In [1]:
notam rec id and launch
merge all past col e to one message column
show how many notams by ear and color code

take topic vecs and create clusters 

SyntaxError: invalid syntax (1355468949.py, line 1)

# Human Annotated NOTAM Analysis

Please note that the data for this analysis is saved locally in a `MatchingLaunchNOTAM_GMUCapstone_20220603` folder that is one level above (a parent folder) to where the [GitHub](https://github.com/DAEN-Team-1/Final "GitHub") files have been saved. This was done because a lot of the data files are too large to be included in the GitHub repoo.

In [ ]:
# imports
import pandas as pd
import numpy  as np
# import bamboolib as bam

Read in data

In [ ]:
# Read in csv (not enough headers for data, so using custom one)
names = ['LAUNCHES_REC_ID', 'NOTAM_REC_ID', 'LowerAltitude', 'UpperAltitude', 'IssueTime', 'StartTime', 'StopTime', 'E_CODE',
         '0_no_header', '1_no_header', '2_no_header', '3_no_header', '4_no_header', '5_no_header', '6_no_header', '7_no_header',
         '8_no_header', '9_no_header', '10_no_header', '11_no_header', '12_no_header', '13_no_header', '14_no_header']

df = pd.read_csv('../MatchingLaunchNOTAM_GMUCapstone_20220603/HumanAnnotatedMatches_SVO_DB_20200127.csv',
                 encoding='UTF-8', on_bad_lines='skip', engine="python", delimiter=',',
                 names=names)               

In [ ]:
# Merge unlabeled columns
cmnt_names = ['0_no_header', '1_no_header', '2_no_header', '3_no_header', '4_no_header', '5_no_header', '6_no_header', '7_no_header',
              '8_no_header', '9_no_header', '10_no_header', '11_no_header', '12_no_header', '13_no_header', '14_no_header']

df['Comment'] = df[cmnt_names].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)

# Replace empyty strings with NaNs
df['Comment'].replace(r'^\s*$', np.nan, regex=True, inplace=True)

df.drop(cmnt_names, axis=1, inplace=True)

In [ ]:
# Drop previous headers
df = df.drop(0)
df.reset_index(inplace=True, drop=True)

In [ ]:
df.head()

Look at df info
* Out of 23 columns, only 8 had headers in the spreadsheet. Need to analyze this to see why. 

In [ ]:
df.info()

Descirbe the df
* Columsn with headers seem to be mostly filled (only `UpperAltitude` and `UpperAltitude` seem to be missing some values). 
* Columns that did not have headers are missing much more data. 

In [ ]:
df.describe()

## Column Analysis

#### LAUNCHES_REC_ID
* Included in NOTAM Analysis CSV, but all values were NaN. 
* Repeats, but none cover more than 4% of the data. 

In [ ]:
df['LAUNCHES_REC_ID']

In [ ]:
df['LAUNCHES_REC_ID'].value_counts()

In [ ]:
df['LAUNCHES_REC_ID'].value_counts(dropna=False, normalize=True).apply(lambda x: '%.5f' % x)

#### NOTAM_REC_ID
* Included in NOTAM Analysis CSV
* Not all entries have some number of characters
* `388267` repeats 3x, while `83024` repeats 2x, all other values occur only once
* No NaNs

In [ ]:
df['NOTAM_REC_ID']

In [ ]:
df['NOTAM_REC_ID'].value_counts(dropna=False)

#### LowerAltitude
* NOTAM Analysis CSV equivalent seems to be `MIN_ALT`
* Convert to floats
* Majority are 0.0, at 75%
    * Could this suggest ground based issues?
* Next 5 categories cover at least 1% of the data
* A couple of NaNs
* Repeats
* Includes chart

In [ ]:
df['LowerAltitude'] = pd.to_numeric(df['LowerAltitude'])

In [ ]:
df['LowerAltitude']

In [ ]:
df['LowerAltitude'].value_counts(dropna=False)

In [ ]:
df['LowerAltitude'].value_counts(dropna=False, normalize=True).apply(lambda x: '%.5f' % x)

In [ ]:
df['LowerAltitude'].value_counts(dropna=False, normalize=True).plot(kind='bar', 
                                                                    title='LowerAltitude Proportion of Data',
                                                                    figsize=(10, 5));

#### UpperAltitude
* NOTAM Analysis CSV equivalent seems to be `MAX_ALT`
* Convert to floats
* Majority are `180000.0`, `999000.0`, `50000.0`, and `NaN`, covering 41%, 33%, 17%, and 6% of the data, respectively.
* No other category covers at least 1% of the data
* Repeats
* Includes chart

In [ ]:
df['UpperAltitude'] = pd.to_numeric(df['UpperAltitude'])

In [ ]:
df['UpperAltitude']

In [ ]:
df['UpperAltitude'].value_counts(dropna=False)

In [ ]:
df['UpperAltitude'].value_counts(dropna=False, normalize=True).apply(lambda x: '%.5f' % x)

In [ ]:
df['UpperAltitude'].value_counts(dropna=False, normalize=True).plot(kind='bar', 
                                                                    title='UpperAltitude Proportion of Data',
                                                                    figsize=(10, 5));

#### IssueTime
* NOTAM Analysis CSV equivalent seems to be `ISSUE_DATE`
* Did not get parsed as dates.
    * Parsed through custom fuction
* All entries from 2016-2020
* Includes chart

In [ ]:
from datetime import datetime
def convert_to_date(row):
    try:
        return datetime.strptime(row, '%Y-%m-%d %H:%M:%S')
    except:
        return row
    
def get_year(row):
    try:
        return row.year
    except:
        return row

def get_month(row):
    try:
        return row.month
    except:
        return row
    
def get_day(row):
    try:
        return row.day
    except:
        return row

In [ ]:
df['IssueTime'] = df['IssueTime'].apply(convert_to_date)

In [ ]:
df['IssueTime'].value_counts(dropna=False)

In [ ]:
df['IssueTime'].value_counts(dropna=False, normalize=True).apply(lambda x: '%.5f' % x)

In [ ]:
df['IssueTime_YEAR']  = df['IssueTime'].apply(get_year)
df['IssueTime_MONTH'] = df['IssueTime'].apply(get_month)
df['IssueTime_DAY']   = df['IssueTime'].apply(get_day)

In [ ]:
df['IssueTime_YEAR'] 

In [ ]:
df['IssueTime_YEAR'].value_counts(dropna=False)

In [ ]:
df['IssueTime_YEAR'].value_counts(dropna=False, normalize=True).apply(lambda x: '%.5f' % x)

In [ ]:
df['IssueTime_YEAR'].value_counts(dropna=False).plot(kind='bar',               
                                                     title='IssueTime_YEAR Proportion of Data',
                                                     figsize=(10, 5));

#### StartTime
* NOTAM Analysis CSV equivalent seems to be `POSSIBLE_START_DATE`
* Did not get parsed as dates.
    * Parsed through custom fuction
* All entries from 2016-2020
* Includes chart

In [ ]:
df['StartTime'] = df['StartTime'].apply(convert_to_date)

In [ ]:
df['StartTime'].value_counts(dropna=False)

In [ ]:
df['StartTime'].value_counts(dropna=False, normalize=True).apply(lambda x: '%.5f' % x)

In [ ]:
df['StartTime_YEAR']  = df['StartTime'].apply(get_year)
df['StartTime_MONTH'] = df['StartTime'].apply(get_month)
df['StartTime_DAY']   = df['StartTime'].apply(get_day)

In [ ]:
df['StartTime_YEAR'] 

In [ ]:
df['StartTime_YEAR'].value_counts(dropna=False)

In [ ]:
df['StartTime_YEAR'].value_counts(dropna=False, normalize=True).apply(lambda x: '%.5f' % x)

In [ ]:
df['StartTime_YEAR'].value_counts(dropna=False).plot(kind='bar',               
                                                     title='StartTime_YEAR Proportion of Data',
                                                     figsize=(10, 5));

#### StopTime
* NOTAM Analysis CSV equivalent seems to be `POSSIBLE_END_DATE`
* Did not get parsed as dates.
    * Parsed through custom fuction
* All entries from 2016-2020
* Includes chart

In [ ]:
df['StopTime'] = df['StopTime'].apply(convert_to_date)

In [ ]:
df['StopTime'].value_counts(dropna=False)

In [ ]:
df['StopTime'].value_counts(dropna=False, normalize=True).apply(lambda x: '%.5f' % x)

In [ ]:
df['StopTime_YEAR']  = df['StopTime'].apply(get_year)
df['StopTime_MONTH'] = df['StopTime'].apply(get_month)
df['StopTime_DAY']   = df['StopTime'].apply(get_day)

In [ ]:
df['StopTime_YEAR'] 

In [ ]:
df['StopTime_YEAR'].value_counts(dropna=False)

In [ ]:
df['StopTime_YEAR'].value_counts(dropna=False, normalize=True).apply(lambda x: '%.5f' % x)

In [ ]:
df['StopTime_YEAR'].value_counts(dropna=False).plot(kind='bar',               
                                                    title='StopTime_YEAR Proportion of Data',
                                                    figsize=(10, 5));

#### ADDED: POSSIBLE_DURATION
* All but one duration are in minutes
* One duratoin lasted for over 97 days
    * Could be an anomally?
* Some repeats
* No entry accounts for more than 4% of the data

In [ ]:
df['POSSIBLE_DURATION'] = df['StopTime'] - df['StartTime']

In [ ]:
df['POSSIBLE_DURATION'] 

In [ ]:
df['POSSIBLE_DURATION'].value_counts(dropna=False)

In [ ]:
df['POSSIBLE_DURATION'].value_counts(dropna=False, normalize=True).apply(lambda x: '%.5f' % x)

#### E_CODE
* NOTAM Analysis CSV equivalent is to be `E_CODE`
* `W497A ACT` represents 16% of the data
* A few other categories represent >1% of the data
* Includes chart

In [ ]:
df['E_CODE']

In [ ]:
df['E_CODE'].value_counts(dropna=False)

In [ ]:
df['E_CODE'].value_counts(dropna=False, normalize=True).apply(lambda x: '%.5f' % x)

In [ ]:
df['E_CODE'].value_counts(dropna=False, normalize=True).nlargest(5).plot(kind='bar',            
                                                                         title='E_CODE Top 5 Types Proportion of Data',           
                                                                         figsize=(10, 5));

#### Comment
* None as biggest category, with 58% of the data
* Second biggest category covers 8% of the data, and the third cover 5%
* No other category covers >1% of the data
* Long strings, codes
* Repeats
* Includes chart

In [ ]:
df['Comment']

In [ ]:
df['Comment'].unique()

In [ ]:
df['Comment'].value_counts(dropna=False)

In [ ]:
df['Comment'].value_counts(dropna=False, normalize=True).apply(lambda x: '%.5f' % x)

In [ ]:
df['Comment'].value_counts(dropna=False, normalize=True).nlargest(5).plot(kind='bar',            
                                                                          title='Comment Top 5 Types Proportion of Data',           
                                                                          figsize=(10, 5));